In [1]:
import requests
import pandas as pd 
import geopandas as gpd
import re
from bs4 import BeautifulSoup

In [2]:
result=requests.get('https://news.ftv.com.tw/AMP/News_Amp.aspx?id=2018A25W0003')

In [3]:
result.text

'\r\n\r\n\r\n<!DOCTYPE html>\r\n\r\n<html ⚡ itemscope itemtype="http://schema.org/WebSite">\r\n<head><meta charset="utf-8" /><meta name="viewport" content="width=device-width,minimum-scale=1,initial-scale=1" /><title>普悠瑪翻車前257句43分鐘對話曝光！ 司機一路求援邊開邊修-民視新聞網</title>\r\n<meta itemprop=\'name\'  Property=\'og:title\' content=\'普悠瑪翻車前257句43分鐘對話曝光！ 司機一路求援邊開邊修-民視新聞網\' />\r\n<meta itemprop=\'description\' property=\'og:description\' content=\'行政院調查小組昨晚（24日）首度確認出軌前4分鐘，司機已經向調度員告知要關掉ATP，政院調查的內容已經打臉台鐵先前說法。然而當天事發普悠瑪6432次通聯紀錄今天曝光，顯示司機員在貢寮、福隆間開始，就已經向調度員報告列車動力異常。司機員與調度員通聯紀錄長達43分鐘的顯示，司機員在21日下午4時5分開始向調度員告知列車動力異常，「動力會自動消失」，要求檢查員上列車檢查，接著與檢查員通話過程討論排除問題，雖然司機員嘗試重新降弓，但列車仍在行駛至大溪再度失去動力。\' />\r\n<meta Property=\'og:type\' content=\'website\' />\r\n<meta itemprop=\'url\' Property=\'og:url\' content=\'https://news.ftv.com.tw/API/MetaInfo.aspx?id=2018A25W0003\'/>\r\n<meta itemprop=\'image\' Property=\'og:image\' content=\'https://newsimg.ftv.com.tw/manasystem/FileData/News/8bca0a3e-1705-447a-a636-a8ad4986fb07.jpg\'/>\

In [4]:
soup = BeautifulSoup(result.text, 'html.parser')

In [5]:
p_count=0
start_flag=False
p_contents=soup.find_all('p')
all_data=[]
location=""
time=""
for p_item in p_contents:
    
    text=str(p_item.text)
    if text =='完整通聯紀錄：':
        start_flag=True
        p_count+=1
        continue
    
    if len(re.findall('地點',text))>0:
        location=text
        continue
        
    if len(text.split(':'))==3 and len(text)==8:
        time=text
        continue
            
    # 處理前幾行
    if start_flag==True:
        if  p_count<4:
            p_count+=1
            time=text[0:8]
            all_data.append([location,time,text[8:]])
        else:
            all_data.append([location,time,text])
all_data

[['（地點：貢寮-福隆）', '16:05:59', '司機員：6432機車有問題，有時候動力會自動的消失，麻煩報告一下調度員'],
 ['（地點：貢寮-福隆）', '16:06:13', '司機員：6432機車有問題，有時候動力會自動消失'],
 ['（地點：貢寮-福隆）', '16:06:23', '司機員：時好時壞'],
 ['（地點：貢寮-福隆）', '16:06:23', '16:06:33司機員：2008'],
 ['（地點：福隆-大里）', '16:07:55', '（車況：1、8主風泵停止運轉）'],
 ['（地點：福隆-大里）', '16:07:55', '司機員：6432您好。'],
 ['（地點：福隆-大里）', '16:07:55', '調度員：6432請問你車子怎麼樣？'],
 ['（地點：福隆-大里）', '16:07:55', '司機員：6432機車動力時好時壞。'],
 ['（地點：福隆-大里）', '16:07:55', '調度員：什麼意思？什麼時好時壞？'],
 ['（地點：福隆-大里）', '16:07:55', '司機員：就是有時候動力會自動切斷。'],
 ['（地點：福隆-大里）', '16:07:55', '調度員：動力會自動切斷？有什麼故障指示燈嗎？'],
 ['（地點：福隆-大里）', '16:07:55', '司機員：故障指示燈，就是1車和8車空壓機那個跳開。'],
 ['（地點：福隆-大里）', '16:07:55', '調度員：空壓機跳開，1車到8車空壓機跳開。'],
 ['（地點：福隆-大里）', '16:07:55', '司機員：對呀，那個應該不會影響。'],
 ['（地點：福隆-大里）', '16:07:55', '調度員：空壓機跳開就是你的空調，就是你的冷氣嘛。對不對？'],
 ['（地點：福隆-大里）', '16:07:55', '司機員：對。但是我的動力有時候就是會切掉就沒有了。'],
 ['（地點：福隆-大里）', '16:07:55', '調度員：這樣子厚，你現在開沒有問題嘛。'],
 ['（地點：福隆-大里）', '16:07:55', '司機員：沒有問題，就是花蓮的時候，列檢可不可以上來看一下。'],
 ['（地點：福隆-大里）', '16:07:55', '調度員：你宜蘭有沒有停？'],


In [6]:
rail=gpd.read_file('data/Rail/Rail.shp',encoding='utf-8')
rail.crs = {'init' :'epsg:3826'} # 避免資料沒設，這邊再重新給一次
rail=rail.to_crs(epsg=4326)
# rail.groupby('RAILNAME').size()
rail=rail[rail['RAILNAME']=='臺鐵宜蘭線']
rail.reset_index(inplace=True)
rail

,index,RAILID,RAILNAME,RAILTYPE,MDATE,DEFINITION,geometry
0,405,C0000000002,臺鐵宜蘭線,2,201311,0,LINESTRING (121.7576447331545 25.1030779555811...
1,406,C0000000003,臺鐵宜蘭線,2,201311,0,LINESTRING (121.7952572783527 25.1131891096044...
2,427,F0000000005,臺鐵宜蘭線,1,201311,0,LINESTRING (121.9330555180089 25.0178933211844...
3,428,F0000000006,臺鐵宜蘭線,1,201311,0,LINESTRING (121.9235661372882 25.0193017021040...
4,429,F0000000007,臺鐵宜蘭線,1,201311,0,LINESTRING (121.9179888746297 25.0207342739527...
5,430,F0000000008,臺鐵宜蘭線,1,201311,0,LINESTRING (121.9179888746297 25.0207342739527...
6,431,F0000000009,臺鐵宜蘭線,1,201311,0,LINESTRING (121.8778615548313 25.0261086718516...
7,432,F0000000010,臺鐵宜蘭線,1,201311,0,LINESTRING (121.8225089869099 25.0639513786485...
8,433,F0000000011,臺鐵宜蘭線,1,201311,0,LINESTRING (121.8330555027739 25.0588452177778...
9,434,F0000000012,臺鐵宜蘭線,2,201311,0,LINESTRING (121.9573398481481 24.9983333751662...


In [7]:
from shapely.geometry import Point
station_pd=pd.read_json('data/Station/Station.json', 'r', encoding='utf-8-sig')
station_pd=station_pd[station_pd['gps']!='']
y = [float(row['gps'].split(',')[0].replace(' ','')) for idx,row in station_pd.iterrows()]
x = [float(row['gps'].split(',')[1].replace(' ','')) for idx,row in station_pd.iterrows()]
geom = [Point(xy) for xy in zip(x, y)]
crs = {'init': 'epsg:4326'}
station = gpd.GeoDataFrame(station_pd, crs=crs, geometry=geom)
station[['Station_Name','geometry']].to_file('output/station.shp')
station

,EnglishAddress,EnglishName,Station_Code3,Station_Code4,Station_EName,Station_Name,gps,住址,網站中文站名,電話,geometry
0,none,NMMST,291,6103,NMMST,海科館,"25.137706, 121.800023",無,海科館站,02-24972033,POINT (121.800023 25.137706)
1,"No.2, Fulong St., Gongliao Dist., New Taipei C...",Fulong,083,1810,Fulong,福隆,"25.015893,121.944659",新北市貢寮區福隆里福隆街2號,福隆站,02-24991800,POINT (121.944659 25.015893)
2,"No.33, Chaoyang St., Gongliao Dist., New Taipe...",Gongliao,084,1809,Gongliao,貢寮,"25.022044,121.908703",新北市貢寮區貢寮里朝陽街33號,貢寮站,02-24941500,POINT (121.908703 25.022044)
3,"No.1, Chaoyang St., Shuangxi Dist., New Taipei...",Shuangxi,085,1808,Shuangxi,雙溪,"25.038544,121.866548",新北市雙溪區新基里朝陽街1號,雙溪站,02-24991800,POINT (121.866548 25.038544)
4,"No.159, Mudan Rd., Shuangxi Dist., New Taipei ...",Mudan,086,1807,Mudan,牡丹,"25.058738,121.851977",新北市雙溪區牡丹路159號,牡丹站,02-24932980,POINT (121.851977 25.058738)
5,"No.1, Yuliao Rd., Ruifang Dist., New Taipei Ci...",Sandiaoling,087,1806,Sandiaoling,三貂嶺,"25.065544,121.822559",新北市瑞芳區碩仁里魚寮路1號,三貂嶺站,02-24977896,POINT (121.822559 25.065544)
6,"No.70, Chailiao Rd., Ruifang Dist., New Taipei...",Houtong,088,1805,Houtong,侯硐,"25.087009,121.827424",新北市瑞芳區光復里柴寮路70號,侯硐站,02-24977747,POINT (121.827424 25.087009)
7,"No.82, Sec. 3, Mingdeng Rd., Ruifang Dist., Ne...",Ruifang,089,1804,Ruifang,瑞芳,"25.108928,121.806149",新北市瑞芳區龍潭里明燈路3段82號,瑞芳站,02-24972033,POINT (121.806149 25.108928)
8,"No.65, Zhongyang Rd., Ruifang Dist., New Taipe...",Sijiaoting,090,1803,Sijiaoting,四腳亭,"25.102751,121.761887",新北市瑞芳區吉慶里中央路65號,四腳亭站,02-24579346,POINT (121.761887 25.102751)
9,"No.51 , Nuannuan Rd. , Nuannuan Dist., Keelung...",Nuannuan,091,1802,Nuannuan,暖暖,"25.102282,121.740329",基隆市暖暖街51號,暖暖站,02-24972033,POINT (121.740329 25.102282)


In [8]:
import folium
m = folium.Map((24.694192,121.775163),zoom_start=18)

folium.GeoJson(
    station.to_json(),
    name='geojson'
).add_to(m)

rail_geojson  = folium.GeoJson(
    rail.to_json(),
    name='geojson').add_to(m)

m

In [9]:
import numpy as np
from shapely.geometry import Point



for i1,r1 in station.iterrows():
    point=r1['geometry']
    min_distance=999
    new_point=point
    for i2,r2 in rail.iterrows():

        line = r2['geometry']

        x = np.array(point.coords[0])

        u = np.array(line.coords[0])
        v = np.array(line.coords[len(line.coords)-1])

        n = v - u
        n /= np.linalg.norm(n, 2)

        P = u + n*np.dot(x - u, n)
        distance=Point(P).distance(point)
        if distance<min_distance:
            min_distance=distance
            new_point=Point(P)
    station.at[i1,'geometry']=new_point
            
    

In [12]:
import folium
m = folium.Map((24.694192,121.775163),zoom_start=18)

folium.GeoJson(
    station.to_json(),
    name='geojson'
).add_to(m)

rail_geojson  = folium.GeoJson(
    rail.to_json(),
    name='geojson',
 
)
popup = folium.Popup('Hi')
popup.add_to(rail_geojson)
rail_geojson.add_to(m)

m

In [13]:
distance

1.2352606642777664